In [3]:
import cv2
import os
import tensorflow as tf
from tensorflow import keras
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import normalize
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense

In [4]:
image_directory = 'Dataset/'

In [6]:
if os.path.exists(os.path.join(image_directory,'no')):
    no_tumor_image = os.listdir(os.path.join(image_directory,'no'))
    
else:
    print("'no/' directory doesn't exist")
    
yes_tumor_image = os.listdir(os.path.join(image_directory,'yes'))
dataset = []
label = []
INPUT_SIZE = 64

In [10]:
for i, image_name in enumerate(no_tumor_image):
    if image_name.split('.')[1] == 'jpg':
        image_path = os.path.join(image_directory,'no',image_name)
        image = cv2.imread(image_path)
        image = Image.fromarray(image,'RGB')  #by default it was BGR
        image = image.resize((INPUT_SIZE, INPUT_SIZE))
        dataset.append(np.array(image))
        label.append(0)

In [12]:
for i, image_name in enumerate(yes_tumor_image):
    if image_name.split('.')[1] == 'jpg':
        image_path = os.path.join(image_directory, 'yes', image_name)
        image = cv2.imread(image_path)
        image = Image.fromarray(image, 'RGB')
        image = image.resize((INPUT_SIZE, INPUT_SIZE))
        dataset.append(np.array(image))
        label.append(1)

In [13]:
print(len(dataset))
print(len(label))

3000
3000


In [14]:
dataset=np.array(dataset)
label=np.array(label)

In [17]:
dataset.shape , label.shape

((3000, 64, 64, 3), (3000,))

In [18]:
x_train, x_test, y_train, y_test = train_test_split(dataset, label, test_size=0.2, random_state=0)

In [19]:
x_train = normalize(x_train, axis=1)
x_test = normalize(x_test, axis=1)

In [21]:
model = Sequential()

model.add(Conv2D(32,(3,3), input_shape=(INPUT_SIZE, INPUT_SIZE, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, (3,3), kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3), kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))    #50% of the neurons will be randomly set to 0 during each training step
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)                    │ (None, 62, 62, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_5 (Activation)            │ (None, 62, 62, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 31, 31, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 29, 29, 32)          │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_6 (Activation)            │ (None, 29, 29, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 14, 14, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 12, 12, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_7 (Activation)            │ (None, 12, 12, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 6, 6, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 2304)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 64)                  │         147,520 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_8 (Activation)            │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │              65 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_9 (Activation)            │ (None, 1)                   │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 176,225 (688.38 KB)

 Trainable params: 176,225 (688.38 KB)

 Non-trainable params: 0 (0.00 B)

In [22]:
model.fit(x_train, y_train, batch_size=16, verbose=1, epochs=10, validation_data=(x_test, y_test), shuffle=False)

Epoch 1/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 6s 21ms/step - accuracy: 0.6375 - loss: 0.6173 - val_accuracy: 0.8033 - val_loss: 0.4432
Epoch 2/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.8268 - loss: 0.4160 - val_accuracy: 0.8283 - val_loss: 0.3702
Epoch 3/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.8721 - loss: 0.3238 - val_accuracy: 0.8450 - val_loss: 0.3231
Epoch 4/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.9078 - loss: 0.2475 - val_accuracy: 0.9350 - val_loss: 0.1883
Epoch 5/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.9432 - loss: 0.1703 - val_accuracy: 0.9500 - val_loss: 0.1504
Epoch 6/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.9623 - loss: 0.1142 - val_accuracy: 0.9517 - val_loss: 0.1333
Epoch 7/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.9773 - loss: 0.0710 - val_accuracy: 0.9650 - val_loss: 0.0954
Epoch 8/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.9890 - loss: 0.0364 - val_accu

In [23]:
model.save('BrainTumor.h5')

In [24]:
predictions = model.predict(x_test)

19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step


In [26]:
# Converting the predictions to binary values (0 or 1) using a threshold (0.5)
binary_predictions = (predictions > 0.5).astype(int)

result = model.evaluate(x_test,y_test)
print("Performance of the model :",result)

19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9618 - loss: 0.1345
Performance of the model : [0.15656505525112152, 0.9616666436195374]
